In [3]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go
import requests
import lightgbm as lgb

# ------------------------------
# 1) Load & prepare historical data
# ------------------------------
# Read the CSV file into a DataFrame
# Please ensure the path to your CSV file is correct
file_path = r"C:\Users\HP 15\Desktop\quiet_data_clean_corrected_2.csv"
quiet_df = pd.read_csv(file_path)

# Cyclical time features
quiet_df['DOY_s'] = np.sin(2 * np.pi * quiet_df['DOY'] / 365)
quiet_df['DOY_c'] = np.cos(2 * np.pi * quiet_df['DOY'] / 365)
quiet_df['HH_s']  = np.sin(2 * np.pi * quiet_df['Hour'] / 24)
quiet_df['HH_c']  = np.cos(2 * np.pi * quiet_df['Hour'] / 24)

# Feature set used by the model
FEATURES = ['DOY_s', 'DOY_c', 'HH_s', 'HH_c', 'Sunspot_No', 'Lat', 'Lon', 'f10_7_index']
X = quiet_df[FEATURES]
y = quiet_df['vTEC_corrected']

# Scale & split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train LightGBM model
lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
print("[LightGBM] [Info] Start training from score", np.mean(y_train))
lgb_model.fit(X_train, y_train)

# ------------------------------
# 2) NOAA helpers (Observed if available, else Predicted)
# ------------------------------
OBS_URL = "https://services.swpc.noaa.gov/json/solar-cycle/observed-solar-cycle-indices.json"
PRED_URL = "https://services.swpc.noaa.gov/json/solar-cycle/predicted-solar-cycle.json"

def _extract_time_tag(row):
    for k in ('time-tag', 'time_tag', 'timeTag', 'time'):
        if k in row and row[k]:
            return str(row[k]).split('T')[0][:7]
    return None

def _get_value(row, keys):
    for k in keys:
        if k in row and row[k] is not None:
            try:
                return float(row[k])
            except (ValueError, TypeError):
                pass
    return None

def fetch_noaa_solar_vars(year: int, month: int):
    ym = f"{int(year):04d}-{int(month):02d}"
    
    # --- Try observed first ---
    try:
        obs = requests.get(OBS_URL, timeout=10)
        if obs.ok:
            for row in obs.json():
                if _extract_time_tag(row) == ym:
                    ssn = _get_value(row, ['ssn', 'sunspot_number', 'sunspotno'])
                    f10 = _get_value(row, ['f10.7', 'f107', 'flux', 'f10.7_adj'])
                    if ssn is not None and f10 is not None:
                        print(f"Fetched observed data for {ym}: SSN={ssn}, F10.7={f10}")
                        return ssn, f10
    except requests.exceptions.RequestException as e:
        print(f"Error fetching observed data: {e}")

    # --- Fall back to predicted ---
    try:
        pred = requests.get(PRED_URL, timeout=10)
        if pred.ok:
            for row in pred.json():
                if _extract_time_tag(row) == ym:
                    ssn = _get_value(row, ['predicted_ssn', 'ssn'])
                    f10 = _get_value(row, ['predicted_f10.7', 'f10.7'])
                    if ssn is not None and f10 is not None:
                        print(f"Fetched predicted data for {ym}: SSN={ssn}, F10.7={f10}")
                        return ssn, f10
    except requests.exceptions.RequestException as e:
        print(f"Error fetching predicted data: {e}")

    # --- Final fallback: historical DOY-mean + SIMULATION ---
    try:
        dt = pd.Timestamp(year=int(year), month=int(month), day=15)
        doy = dt.dayofyear
        avg = quiet_df.groupby('DOY')[['Sunspot_No', 'f10_7_index']].mean()
        if doy in avg.index:
            row = avg.loc[doy]
            ssn_fallback = float(row['Sunspot_No'])
            f10_fallback = float(row['f10_7_index'])
            
            if pd.isna(ssn_fallback) or pd.isna(f10_fallback):
                 print(f"Warning: Fallback produced NaN for DOY {doy}. Using hardcoded values.")
            else:
                print(f"Warning: Using historical DOY-mean fallback for {ym}.")
                
                # -----------------------------------------------------------------
                # --- NEW: SOLAR CYCLE SIMULATION FOR FUTURE YEARS ---
                # This part adds a simulated ~11-year cycle for future years.
                # Solar Cycle 26 is predicted to peak around mid-2034.
                cycle_length_years = 11.0
                peak_year = 2034.5 # Mid-2034
                
                # Calculate a scaling factor (from ~0.5 to 1.5) based on a cosine wave
                # This wave peaks in the 'peak_year'
                cycle_phase = (year + month / 12.0 - peak_year) * (2 * np.pi / cycle_length_years)
                scaling_factor = 0.5 * np.cos(cycle_phase) + 1.0 # Varies between 0.5 and 1.5

                # Apply the scaling factor to the historical averages
                simulated_ssn = ssn_fallback * scaling_factor
                simulated_f10 = f10_fallback * scaling_factor
                
                print(f"Applied simulation factor of {scaling_factor:.2f}. New values: SSN={simulated_ssn:.1f}, F10.7={simulated_f10:.1f}")
                return simulated_ssn, simulated_f10
                # --- END OF NEW SIMULATION CODE ---
                # -----------------------------------------------------------------
                
    except Exception as e:
        print(f"Error in historical fallback: {e}")

    # Hard fallback if all else fails
    print("Warning: Using hardcoded fallback values.")
    return 50.0, 100.0


# ------------------------------
# 3) Dash app UI
# ------------------------------
app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H1("Daily vTEC Prediction (NOAA-driven SSN & F10.7)"),
    html.Div([
        html.Div([
            html.Label("Year"),
            dcc.Input(id='input_year', type='number', value=2031),
        ]),
        html.Div([
            html.Label("Month"),
            dcc.Input(id='input_month', type='number', value=1),
        ]),
        html.Div([
            html.Label("Day"),
            dcc.Input(id='input_day', type='number', value=1),
        ]),
    ], style={"display": "flex", "gap": "16px", "marginBottom": "12px"}),

    html.Button('Predict Daily vTEC', id='predict_button', n_clicks=0, style={"marginTop": "12px"}),
    html.Div(id='noaa_values', style={"marginTop": "8px", "fontStyle": "italic"}),
    dcc.Graph(id='daily_pred_plot', style={"height": "520px"}),
])

# ------------------------------
# 4) Prediction callback
# ------------------------------
@app.callback(
    Output('daily_pred_plot', 'figure'),
    Output('noaa_values', 'children'),
    Input('predict_button', 'n_clicks'),
    State('input_year', 'value'),
    State('input_month', 'value'),
    State('input_day', 'value')
)
def predict_daily_vtec(n_clicks, year, month, day):
    if n_clicks == 0:
        return go.Figure(), "Enter a date and click 'Predict Daily vTEC' to begin."

    sunspot, f107 = fetch_noaa_solar_vars(int(year), int(month))

    try:
        date = pd.Timestamp(int(year), int(month), int(day))
    except ValueError:
        return go.Figure(layout={'title': 'Invalid Date'}), f"Error: The date {year}-{month}-{day} is not valid."

    days_in_year = 366 if date.is_leap_year else 365
    doy = date.dayofyear
    hours = np.arange(0, 24)
    preds = []

    for h in hours:
        row_df = pd.DataFrame({
            'DOY_s': [np.sin(2 * np.pi * doy / days_in_year)],
            'DOY_c': [np.cos(2 * np.pi * doy / days_in_year)],
            'HH_s':  [np.sin(2 * np.pi * h / 24)],
            'HH_c':  [np.cos(2 * np.pi * h / 24)],
            'Sunspot_No': [sunspot],
            'Lat': [38.0],
            'Lon': [9.0],
            'f10_7_index': [f107]
        })
        row_df = row_df[FEATURES]
        pred = lgb_model.predict(scaler.transform(row_df))[0]
        preds.append(pred)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hours, y=preds, mode='lines+markers', name='Predicted vTEC'))
    max_idx = int(np.argmax(preds))
    fig.add_trace(go.Scatter(
        x=[hours[max_idx]], y=[preds[max_idx]], mode='markers+text',
        marker=dict(size=12, color='crimson'),
        text=[f'Max: {preds[max_idx]:.2f}'], textposition='top center', name='Max vTEC'))
    fig.update_layout(
        title=f"Predicted Daily vTEC for {date.strftime('%Y-%m-%d')}",
        xaxis_title='Hour (UTC)', yaxis_title='vTEC_corrected')

    note = (
        f"NOAA SSN={sunspot:.1f}, F10.7={f107:.1f} for {int(year):04d}-{int(month):02d} "
        f"(observed if available, else predicted/fallback)."
    )
    return fig, note

if __name__ == '__main__':
    app.run(debug=True, port=8090)

[LightGBM] [Info] Start training from score 21.906338183200916
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 983
[LightGBM] [Info] Number of data points in the train set: 49967, number of used features: 8
[LightGBM] [Info] Start training from score 21.906338
